In [2]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [3]:
using StaticArrays, DifferentialEquations, DynamicalSystems, ForwardDiff
E, x, y  = 0..20, 0..20,  0..20
box = E × x × y
using CairoMakie, GLMakie;

In [4]:
function RMA(u, p, t)
    x,y,z = u
    r, k, a1, b1, a2, b2, d1, d2 = p
    
   ẋ = x * ( r * ( 1 - x/k ) - a1 * y / ( 1 + b1 * x ) )
   ẏ = y * ( a1 * x / ( 1 + b1 * x ) - a2 * z / ( 1 + b2 * y ) - d1 )
   ż = z * ( a2 * y / ( 1 + b2 * y ) - d2 )
    return SVector(ẋ , ẏ, ż)
end

RMA (generic function with 1 method)

In [5]:
function jac_RMA(u, p, t)
    x,y,z = u
    r, k, a1, b1, a2, b2, d1, d2 = p

    fx = x * ( -r/k + a1 *y * b1 / (b1 * x + 1)^2 ) + r * (1 - x/k) - a1*y/(1+b1*x) 
    gy = -a2 * z / ( b2 * y + 1 ) + a2 * b2 * z * y / ( b2 * y + 1 )^2 + a1 * x / (b1 * x + 1) - d1
    hz = a2 * y / ( 1 + b2 * y ) - d2

    fy = -a1 * x / (1 + b1 * x)
    gz = -a2 * y / (1 + b2 * y)

    gx = y * a1 / ( b1 * x + 1 )^2
    hy = z * a2 / ( b2 * y + 1 )^2

    return SMatrix{3, 3}( fx, gx, 0.0,
                    fy, gy, hy,
                    0.0, gz, hz )
end

jac_RMA (generic function with 1 method)

In [6]:
a1 = 5; a2 = 0.1; b1 = 3; b2 = 2; d1 = 0.4; d2 = 0.01

0.01

In [7]:
r = 1.1
k = 1.09841
p = [r, k, a1, b1, a2, b2, d1, d2]

8-element Vector{Float64}:
 1.1
 1.09841
 5.0
 3.0
 0.1
 2.0
 0.4
 0.01

In [8]:
u0 = [0.7297732151428117, 0.1889267215901415, 9.585811098760479]

3-element Vector{Float64}:
 0.7297732151428117
 0.1889267215901415
 9.585811098760479

In [9]:
t = 5000; tt = 10000;

In [10]:
Jacobi(u0) = ForwardDiff.jacobian((x) -> RMA(x, p, t), u0)

Jacobi (generic function with 1 method)

In [11]:
Jacobi(u0)

3×3 Matrix{Float64}:
 -0.454527   -1.14409    0.0
  0.0928683   0.239169  -0.0137117
  0.0         0.50492    0.00371167

In [12]:
jac_RMA(u0, p, 0)

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 -0.454527   -1.14409    0.0
  0.0928683   0.239169  -0.0137117
  0.0         0.50492    0.00371167

In [13]:
integrator_setting = (alg = RK4(), adaptive = false, dt = 0.01);

In [14]:
ds = CoupledODEs(RMA, u0, p, diffeq = integrator_setting)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  RMA
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.01)
 parameters:    [1.1, 1.09841, 5.0, 3.0, 0.1, 2.0, 0.4, 0.01]
 time:          0.0
 state:         [0.7297732151428117, 0.1889267215901415, 9.585811098760479]


In [15]:
fp, eigs, _ = fixedpoints(ds, box, jac_RMA)

┌ Warning: Non-unique fixed points found!
└ @ ChaosTools C:\Users\Alex\.julia\packages\ChaosTools\VMXUY\src\stability\fixedpoints.jl:104


(3-dimensional StateSpaceSet{Float64} with 4 points, Vector{ComplexF64}[[-0.3999999999999993 + 0.0im, -0.009999999999999962 + 0.0im, 1.0999999999999979 + 0.0im], [-0.7575582956475049 + 0.0im, 0.054359640844178256 - 0.06457997332491508im, 0.054359640844178256 + 0.06457997332491508im], [0.007180090605149502 + 0.0im, 0.06664237062279976 - 0.5458135880836007im, 0.06664237062279976 + 0.5458135880836007im], [-1.1 + 0.0im, -0.009999999999999998 + 0.0im, 0.8786393278124806 + 0.0im]], Bool[0, 0, 0, 0])

In [16]:
fp

3-dimensional StateSpaceSet{Float64} with 4 points
 1.54732e-16  3.74718e-16   2.83606e-16
 0.934299     0.125        10.3551
 0.105263     0.261733      6.69894e-17
 1.09841      2.5599e-17    1.40695e-16

In [17]:
eigs[2]

3-element Vector{ComplexF64}:
  -0.7575582956475049 + 0.0im
 0.054359640844178256 - 0.06457997332491508im
 0.054359640844178256 + 0.06457997332491508im

In [18]:
tr, trange = trajectory(ds, t; Δt = integrator_setting.dt, Ttr = tt)

(3-dimensional StateSpaceSet{Float64} with 500001 points, 10000.0:0.01:15000.0)

In [31]:
f = Figure(resolution = (1600, 600))
ax = Axis3(f[1, 1])
lines!(ax, tr[:, 1], tr[:, 2], tr[:, 3])
idxfp = 2
scatter!(ax, fp[idxfp][1], fp[idxfp][2], fp[idxfp][3], color = :deeppink)
display(f)

GLMakie.Screen(...)